In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from config.config import OUTPUT_CSV_FILE, CLEANED_CSV_FILE

sns.set(style="whitegrid")

# Load data
df = pd.read_csv(CLEANED_CSV_FILE)
print("Data loaded successfully.\n")